In [1]:
import requests
import json
import numpy as np
import pandas as pd

In [3]:
from config import gkey

In [4]:
csv_path = "Project1_AmazonSites.csv"
city_csv = pd.read_csv(csv_path)
# to remove all null values
city_df = city_csv.iloc[:39,:9]
# change zip to only integer
city_df['zip code'] = city_df['zip code'].astype(int)
for index, zip in city_df['zip code'].iteritems():
    if len(str(zip)) == 4:
        city_df['zip code'].iloc[index] = "0"+str(zip)
city_df.head()

C:\Users\hleao\AppData\Local\Continuum\anaconda3\envs\Python\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,amazon city,site,site name,address,city,state,zip code,latitude,longitude
0,Northern Virginia Area,Site1,Center for Innovation Technology,2214 Rock Hill Road,Herndon,VA,20170,38.96,-77.42
1,Northern Virginia Area,Site2,Crystal City and Potomac Yard,3535 S Ball Street,Arlington,VA,22202,38.84,-77.05
2,Washington DC,Site1,Nationals Park/ D.C United Stadium,32-60 R Street,Washington,DC,20024,38.86,-77.01
3,Washington DC,Site2,Union Station,10 G Street,Washington,DC,20002,38.89,-77.00
4,Washington DC,Site3,Hill-East Neighborhood,1430 Pennsylvania Avenue,Washington,DC,20003,38.88,-76.97


In [20]:
city_param = city_df[['latitude', 'longitude', 'address','site name','city','amazon city']].head()
city_param

,latitude,longitude,address,site name,city,amazon city
0,38.96,-77.42,2214 Rock Hill Road,Center for Innovation Technology,Herndon,Northern Virginia Area
1,38.84,-77.05,3535 S Ball Street,Crystal City and Potomac Yard,Arlington,Northern Virginia Area
2,38.86,-77.01,32-60 R Street,Nationals Park/ D.C United Stadium,Washington,Washington DC
3,38.89,-77.00,10 G Street,Union Station,Washington,Washington DC
4,38.88,-76.97,1430 Pennsylvania Avenue,Hill-East Neighborhood,Washington,Washington DC


In [23]:
airport_list = []

## Airports
for city in city_param.values:
    target_coordinates = f"{city[0]}, {city[1]}"
    #radius of within 31 miles or 50000 meters
    target_radius = 50000
    target_type = "airport"
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    airport_json = response.json()
    for airport_num in range(len(airport_json['results'])):
        airport_str = airport_json['results'][airport_num]['name']
        try:
            airport_rating = airport_json['results'][airport_num]['rating']
        except:
            airport_rating = None
        airport_list.append({'Site Name':city[3],
                     'City':city[4],
                     'Amazon City':city[5],
                     'Airport':airport_str,
                     'Airport Rating':airport_rating})
airport_df = pd.DataFrame(airport_list)
    

In [36]:
amazon_city_group = airport_df.groupby(['Amazon City','City','Site Name','Airport'])
final_amazon_city_df = amazon_city_group.mean()
final_amazon_city_df

Airport Rating
Amazon City            City       Site Name                          Airport                                                           
Northern Virginia Area Arlington  Crystal City and Potomac Yard      A2 Global Shipping                                             5.0
                                                                     Anacostia Helicopter Facility                                  5.0
                                                                     Ball Aerospace & Technologies                                  NaN
                                                                     Baltimore/Washington International Thurgood Mar...             4.0
                                                                     College Park Airport - CGS                                     4.9
                                                                     Davison AAF                                                    4.4
                                                                     Dulles International Airport                                   3.9
                                                                     Freeway Airport Inc                                            3.7
                                                                     MD83                                                           4.0
                                                                     Manassas Regional Airport                                      4.1
                                                                     Maryland Airport-2W5                                           3.7
                                                                     Montgomery County Airpark                                      4.3
                                                                     Norwood                                                        NaN
                                                                     Potomac Airfield                                               4.8
                                                                     Ronald Reagan Washington National Airport                      3.9
                                                                     Tipton Airport                                                 5.0
                                                                     Washington Executive Airpark/Hyde Field                        4.2
                       Herndon    Center for Innovation Technology   A2 Global Shipping                                             5.0
                                                                     Anacostia Helicopter Facility                                  5.0
                                                                     Ball Aerospace & Technologies                                  NaN
                                                                     College Park Airport - CGS                                     4.9
                                                                     Davison AAF                                                    4.4
                                                                     Dulles International Airport                                   3.9
                                                                     Dulles Jet Center                                              4.2
                                                                     Ijamsville Airport                                             NaN
                                                                     Leesburg Executive Airport                                     4.2
                                                                     Manassas Regional Airport                                      4.1
                                                                     Montgomery County Airpark                                      4.3
                                         